<a href="https://colab.research.google.com/github/data-hound/talktopapers/blob/master/TalkToPapers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This colab lets you upload a paper to your drive and talk to it using Open AI's embeddings. 



In [ ]:
#@title Install Dependencies
!pip install pypdf
!pip install wget
!pip install PyPDF2
!pip install tiktoken
!pip install openai

In [2]:
#@title Import Dependencies
import sys
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib import patches
import argparse
from pypdf import PdfReader
from pathlib import Path
import requests
from google.colab import drive
import pandas as pd
import numpy as np
import openai 
import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

## Setup
Specify api key, mount Gdrive

In [ ]:
#@title Setup
drive.mount('/content/drive')
openai.api_key = 'sk-somerandomcombination0falpanumerics' #@param {type:"string"}
sys.path.append("../")

In [36]:
#@title #Download Paper

#@markdown 

#@markdown #####**Paper URL:**

# ! wget https://export.arxiv.org/pdf/2104.08894.pdf 

url = 'http://export.arxiv.org/pdf/1706.03762.pdf'#@param {type:"string"}
verbose = True #@param {type:"boolean"}

import subprocess

def runcmd(cmd, verbose = False, *args, **kwargs):

    process = subprocess.Popen(
        cmd,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        text = True,
        shell = True
    )
    std_out, std_err = process.communicate()
    if verbose:
        print(std_out.strip(), std_err)
    pass

filename = '/content/drive/MyDrive/' + 'NeurIPS-2020-deep-graph-pose-a-semi-supervised-deep-graphical-model-for-improved-animal-pose-tracking-Paper.pdf'

if 'MyDrive' not in url:
  runcmd("wget {}".format(url), verbose=verbose)

  filename = '/content/' + url.split('/')[-1]
else:
  filename = url

import os
print("Checking filename exists...")
assert os.path.exists(filename), "File does not exist in the given location"
print("File Exists")

 --2023-01-16 05:14:13--  http://export.arxiv.org/pdf/1706.03762.pdf
Resolving export.arxiv.org (export.arxiv.org)... 128.84.21.203
Connecting to export.arxiv.org (export.arxiv.org)|128.84.21.203|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2201700 (2.1M) [application/pdf]
Saving to: ‘1706.03762.pdf’

     0K .......... .......... .......... .......... ..........  2% 76.0M 0s
    50K .......... .......... .......... .......... ..........  4% 1.36M 1s
   100K .......... .......... .......... .......... ..........  6% 1.41M 1s
   150K .......... .......... .......... .......... ..........  9% 65.5M 1s
   200K .......... .......... .......... .......... .......... 11%  215M 1s
   250K .......... .......... .......... .......... .......... 13% 1.38M 1s
   300K .......... .......... .......... .......... .......... 16% 80.5M 1s
   350K .......... .......... .......... .......... .......... 18% 1.43M 1s
   400K .......... .......... .......... .......... .........

## Parse PDF to text

In [14]:
def parse_paper(path):
  print("Parsing paper")
  reader = PdfReader(path)
  number_of_pages = len(reader.pages)
  print(f"Total number of pages: {number_of_pages}")
  paper_text = []
  for i in range(number_of_pages):
    page = reader.pages[i]
    page_text = []

    def visitor_body(text, cm, tm, fontDict, fontSize):
      x = tm[4]
      y = tm[5]
      # ignore header/footer
      if (y > 50 and y < 720) and (len(text.strip()) > 1):
        page_text.append({
          'fontsize': fontSize,
          'text': text.strip().replace('\x03', ''),
          'x': x,
          'y': y
        })

    _ = page.extract_text(visitor_text=visitor_body)

    blob_font_size = None
    blob_text = ''
    processed_text = []

    for t in page_text:
      if t['fontsize'] == blob_font_size:
        blob_text += f" {t['text']}"
      else:
        if blob_font_size is not None and len(blob_text) > 1:
          processed_text.append({
            'fontsize': blob_font_size,
            'text': blob_text,
            'page': i
          })
        blob_font_size = t['fontsize']
        blob_text = t['text']
    paper_text += processed_text
  return paper_text

In [37]:
paper_text = parse_paper(filename)

Parsing paper
Total number of pages: 15


In [38]:
assert not (paper_text == []), "Paper Text should be non-empty"

## Apply a small filter

In [39]:
filtered_paper_text = []
for row in paper_text:
  if len(row['text']) < 30:
    continue
  filtered_paper_text.append(row)

## Convert to dataframe and inspect

In [40]:
df = pd.DataFrame(filtered_paper_text)
print(df.shape)
df.head()


(43, 3)


,fontsize,text,page
0,9.9626,Ashish Vaswani Google Brain avaswani@google.co...,0
1,9.9626,University of Toronto aidan@cs.toronto.edu Łuk...,0
2,9.9626,The dominant sequence transduction models are ...,0
3,9.9626,"Recurrent neural networks, long short-term mem...",0
4,8.9664,Equal contribution. Listing order is random. J...,0


## Calculate pdf embeddings

In [ ]:
embedding_model = "text-embedding-ada-002"
embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
df["embeddings"] = embeddings

In [21]:
df.shape

(36, 4)

## Embed query and Search

We return the chunk in pdf with highest cosine similarity with query embedding

In [20]:
def search_reviews(df, query, n=3, pprint=True):
    query_embedding = get_embedding(
        query,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        
    )
    return results

## Few Example Results

In [23]:
#@title Query!

#@markdown 

#@markdown #####**Query:**
query = "What embeddings were used" #@param {type:"string"}
n=3 #@param {type:'integer'}

results = search_reviews(df, query=query, n=3)
results.iloc[0]['text']


'We also experimented with using learned positional embeddings [ 8] instead, and found that the two versions produced nearly identical results (see Table 3 row (E)). We chose the sinusoidal version because it may allow the model to extrapolate to sequence lengths longer than the ones encountered during training.'

In [55]:
results = search_reviews(df, "explain how multi head self attention works", n=3)
results.iloc[0]['text']


'The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [20], ByteNet [ 15] and ConvS2S [ 8], all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes it more difﬁcult to learn dependencies between distant positions [ 11]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the se

In [ ]:
results = search_reviews(df, "explain the training procedure", n=3)
results.iloc[0]['text']

'This section describes the training regime for our models. 5.1 Training Data and Batching We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million sentence pairs. Sentences were encoded using byte-pair encoding [ 3], which has a shared source- target vocabulary of about 37000 tokens. For English-French, we used the signiﬁcantly larger WMT 2014 English-French dataset consisting of 36M sentences and split tokens into a 32000 word-piece vocabulary [ 38]. Sentence pairs were batched together by approximate sequence length. Each training batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000 target tokens. 5.2 Hardware and Schedule We trained our models on one machine with 8 NVIDIA P100 GPUs. For our base models using the hyperparameters described throughout the paper, each training step took about 0.4 seconds. We trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the b